In [1]:
import os
import json
import re

In [ ]:
%ls

In [2]:
def find_json(root):
    results = []
    for file in os.listdir(root):
        if file == 'result':
            continue
        filepath = os.path.join(root, file)
        if(file.split('.')[-1] == 'json'):
            results.append(filepath)
        if os.path.isdir(filepath):
            results.extend(find_json(filepath))
    return results

In [3]:
filepaths = find_json('.')
filepaths

['./한국어_SNS 3/Training/[라벨]한국어SNS_train/식음료.json',
 './개인및관계-002.json',
 './한국어_SNS 2/Training/[라벨]한국어SNS_train/상거래(쇼핑).json',
 './한국어_SNS 2/Training/[라벨]한국어SNS_train/미용과건강.json',
 './한국어_SNS 2/Training/[라벨]한국어SNS_train/시사교육.json',
 './한국어_SNS 2/Training/[라벨]한국어SNS_train/행사.json',
 './한국어_SNS 2/Training/[라벨]한국어SNS_valid/여가생활.json',
 './한국어_SNS 2/Training/[라벨]한국어SNS_valid/개인및관계.json',
 './한국어_SNS 2/Validation/[라벨]한국어SNS_valid/상거래(쇼핑).json',
 './한국어_SNS 2/Validation/[라벨]한국어SNS_valid/식음료.json',
 './한국어_SNS 2/Validation/[라벨]한국어SNS_valid/미용과건강.json',
 './한국어_SNS 2/Validation/[라벨]한국어SNS_valid/여가생활.json',
 './한국어_SNS 2/Validation/[라벨]한국어SNS_valid/시사교육.json',
 './한국어_SNS 2/Validation/[라벨]한국어SNS_valid/일과직업.json',
 './한국어_SNS 2/Validation/[라벨]

In [4]:
with open(filepaths[0], 'r') as f:
    data = json.load(f)

In [5]:
def extract_participants(data):
    participant = {}
    for i, info in enumerate(data['header']['participantsInfo']):
        participant[info['participantID']] = f'화자{i + 1}'
    return participant

In [6]:
def regex(uttr):
    uttr = re.sub(r'#@시스템#사진#', '', uttr)
    uttr = re.sub(r'#@[가-힣]+#', '', uttr)
    uttr = re.sub(r'\.+', '...', uttr)
    return uttr

In [7]:
def extract_conversation(data, i):
    participant = extract_participants(data['data'][i])
    conversation = ''
    for utterance in data['data'][i]['body']:
        uttr = regex(utterance['utterance'])
        if uttr == '':
            continue
        conversation += f"{participant[utterance['participantID']]}: {uttr}\n"
    return conversation

In [8]:
HEADER_INDEX = "sns_"
global_index = 0
sns_data = {'data' : []}

In [9]:
def extract(file):
    global global_index, sns_data
    print("data loading: ", file)
    with open(file, 'r') as f:
        data = json.load(f)
    print("data loaded: ", file)
    for i in range(data['numberOfItems']):
        res = extract_conversation(data, i)
        if(res == ''):
            continue
        sns_data['data'].append({
            '_id' : HEADER_INDEX + str(global_index),
            'text' : res
        })
        global_index += 1
    print("data extraction complete: ", file)
    print("global index: ", global_index)

In [10]:
for file in filepaths:
    extract(file)

data loading:  ./한국어_SNS 3/Training/[라벨]한국어SNS_train/식음료.json
data loaded:  ./한국어_SNS 3/Training/[라벨]한국어SNS_train/식음료.json
data extraction complete:  ./한국어_SNS 3/Training/[라벨]한국어SNS_train/식음료.json
global index:  146620
data loading:  ./개인및관계-002.json
data loaded:  ./개인및관계-002.json
data extraction complete:  ./개인및관계-002.json
global index:  658116
data loading:  ./한국어_SNS 2/Training/[라벨]한국어SNS_train/상거래(쇼핑).json
data loaded:  ./한국어_SNS 2/Training/[라벨]한국어SNS_train/상거래(쇼핑).json
data extraction complete:  ./한국어_SNS 2/Training/[라벨]한국어SNS_train/상거래(쇼핑).json
global index:  773619
data loading:  ./한국어_SNS 2/Training/[라벨]한국어SNS_train/미용과건강.json
data loaded:  ./한국어_SNS 2/Training/[라벨]한국어SNS_train/미용과건강.json
data extraction complete:  ./한국어_SNS 2/Training/[라벨]한국어SNS_train/미용과건강.json
global index:  873002
data loading:  ./하

In [ ]:
sns_data['data'][1000000]

In [11]:
with open('result/sns_data.json', 'w') as f:
    json.dump(sns_data, f, ensure_ascii=False)